# Delete Sales Orders
In order to remove test data, we want to delete all sales orders of a given year.


API_SALES_ORDER_SRV was activated. For details on OData see `ImportPrices.iynb`.

Added privileges to role `ZUCC_ANALYTICS_ODATA`:
| Auth object | Field | Value |  |
|-------------|-------|-------|--|
| V_VBAK_AAT  | AUART | OR | Standard order
|             | ACTVT    | *
| V_VBAK_VKO  | VKORG    | *    | Sales org
|             | VTWEG    | *    |
|             | SPART    | *    |
|             | ACTVT    | *
| V_VBAK_CCD  | AUART | OR | Standard order
|             | ACTVT    | *

In [22]:
import requests
import pyodata
import Credentials # local modules must start with capital letter
from importlib import reload
reload(Credentials) # just in case; be able to reload here without restarting the kernel

<module 'Credentials' from 'c:\\Users\\verbarg\\Nextcloud\\Forschung\\2022 Reporting UCC\\2022-12 GeneratorGB\\gb-salesdata\\abap\\Credentials.py'>

In [23]:
# To get more logging from pyodata use logging.DEBUG
import logging

logging.basicConfig()
root_logger = logging.getLogger()
root_logger.setLevel(logging.ERROR)

Connect to OData service

In [24]:
service_url = f'{Credentials.p_host}:443/sap/opu/odata/sap/API_SALES_ORDER_SRV/'
session = requests.Session()
session.auth = (Credentials.p_username, Credentials.p_password)
session.params = {'sap-client': Credentials.p_client}   # not necessary, client is already defined in the service
# CSRF Token
response = session.head(service_url, headers={'x-csrf-token': 'fetch'})
token = response.headers.get('x-csrf-token', '')
session.headers.update({'x-csrf-token': token})
# service
orders_service = pyodata.Client(service_url, session)

Read all sales orders

In [25]:
orders_select = (
    orders_service.entity_sets.A_SalesOrder
    .get_entities()
    .select('SalesOrder, SalesOrderDate')
    #.filter('year(SalesOrderDate) eq 2015')  -- error: Function year is not supported
)
orders = orders_select.execute()

In [26]:
# how many sales orders do we have?
count = {}
for order in orders:
    y = order.SalesOrderDate.year
    if y in count:
        count[y] += 1
    else:
        count[y] = 1
count

{2021: 3}

so sieht ein ETag aus

In [27]:
print(orders[1].etag)
print(orders[1].SalesOrder)

W/"datetimeoffset'2021-08-23T14%3A29%3A56.8285730Z'"
2


and delete them

In [28]:
for order in orders:
    # die ersten drei Aufträge aus 2021 kann man nicht löschen, weil schon Folgebelege
    if order.SalesOrder in ('1','2','3'):
        continue
    if order.SalesOrderDate.year != 2021:  # filter for year hardcoded
        continue
    print(f'Delete {order.SalesOrder}')
    # provide Etag for optimistic concurrency control; otherwise DELETE returns http status 428
    session.headers.update({'if-match' : order.etag})
    orders_service.entity_sets.A_SalesOrder.delete_entity(order.SalesOrder).execute()
